In [10]:
import pandas as pd

YEAR = 2021
year_range = [2021,2022,2023,2024,2025]

base_url = f"https://www.espn.com/nba/salaries/_/year/{YEAR}"

dfs = []

for year in year_range:
    for i in range(1, 15):
        url = base_url if i == 1 else f"{base_url}/page/{i}"
        tmp = pd.read_html(url)[0]
        tmp["year"] = str(year)            
        dfs.append(tmp)

    tables = pd.concat(dfs, ignore_index=True)


In [11]:
tables

,0,1,2,3,year
0,RK,NAME,TEAM,SALARY,2021
1,1,"Stephen Curry, G",Golden State Warriors,"$43,006,362",2021
2,2,"Russell Westbrook, G",Washington Wizards,"$41,358,814",2021
3,3,"Chris Paul, G",Phoenix Suns,"$41,358,814",2021
4,4,"James Harden, G",Brooklyn Nets,"$41,254,920",2021
...,...,...,...,...,...
3055,552,"Elijah Bryant, SG",Milwaukee Bucks,"$24,611",2025
3056,553,"Caleb Homesley, SG",Washington Wizards,"$20,000",2025
3057,554,"Charlie Brown Jr., G",Oklahoma City Thunder,"$19,804",2025
3058,555,"Omer Yurtseven, C",Utah Jazz,"$18,458",2025


In [ ]:
# Can drop rank col
tables.drop(columns=tables.columns[0], inplace=True)

In [13]:
tables.to_markdown("clean/nba_salaries_2021_to_2025.csv")